In [1]:
import os
import warnings
import joblib
import matplotlib
import pandas as pd
import seaborn as sns
import torch
import ttach as tta
import optuna
import pytorch_lightning as pl
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from tqdm.auto import tqdm
from torch.nn import functional as F
from models import *
from resnest101 import *
from se_resnext101 import *
from efficientnetb3 import *
from efficientnetb4 import *
from efficientnetb5 import *
from efficientnetb6 import *
from efficientnetb7 import *
from config import *
from util import *
sns.set()
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
rc = {"figure.figsize" : (9, 7),
      "axes.spines.left" : False,
      "axes.spines.right" : False,
      "axes.spines.bottom" : False,
      "axes.spines.top" : False,
      "xtick.bottom" : True,
      "xtick.labelbottom" : True,
      "ytick.labelleft" : True,
      "ytick.left" : True,
      "axes.grid" : False}
plt.rcParams.update(rc)


def get_model(model_name, params):
    if model_name == 'se_resnext101':
        model = SE_ResNext101(data_dir=data_dir, weighted_sampling=True, batch_size=params['batch_size'], learning_rate=params['learning_rate'])
        return model
    if model_name == 'resnest101':
        model = ResNest101(data_dir=data_dir, weighted_sampling=True, batch_size=params['batch_size'], learning_rate=params['learning_rate'])
        return model
    if model_name == 'efficientnetb3':
        model = EfficientNetB3(data_dir=data_dir, weighted_sampling=True, batch_size=params['batch_size'], learning_rate=params['learning_rate'])
        return model
    if 'efficientnetb4' in model_name:
        model = EfficientNetB4(data_dir=data_dir, weighted_sampling=True, batch_size=params['batch_size'], learning_rate=params['learning_rate'])
        return model
    if 'efficientnetb5' in model_name:
        model = EfficientNetB5(data_dir=data_dir, weighted_sampling=True, batch_size=params['batch_size'], learning_rate=params['learning_rate'])
        return model
    if 'efficientnetb6' in model_name:
        model = EfficientNetB6(data_dir=data_dir, weighted_sampling=True, batch_size=params['batch_size'], learning_rate=params['learning_rate'])
        return model
    if 'efficientnetb7' in model_name:
        model = EfficientNetB7(data_dir=data_dir, weighted_sampling=True, batch_size=params['batch_size'], learning_rate=params['learning_rate'])
        return model

In [ ]:
%%time

model_names = ['efficientnetb3', 'efficientnetb4.1', 'efficientnetb4.2', 'efficientnetb4.3', 'efficientnetb4.4', 'efficientnetb4.5', 
               'efficientnetb5.1', 'efficientnetb5.2', 'efficientnetb5.3', 'efficientnetb5.4', 'efficientnetb6.1', 'efficientnetb6.1', 
               'efficientnetb6.2', 'efficientnetb6.3', 'efficientnetb7.1', 'efficientnetb7.2', 'efficientnetb7.3', 'resnest101', 'se_resnext101']

model_names = ['efficientnetb3']

for model_name in tqdm(model_names):
    
    print(f"===={model_name.upper()}====")

    best_accuracy = 0

    def objective(trial):
        torch.cuda.empty_cache()
        
        params = {
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 3e-4), 
            'batch_size': trial.suggest_categorical('batch_size', [16, 32])
        }
        
        #model = get_model(model_name, params)
        model = EfficientNetB3(data_dir=data_dir, weighted_sampling=True, batch_size=params['batch_size'], learning_rate=params['learning_rate'])

        trainer = pl.Trainer(max_epochs=15, devices=1, accelerator="gpu", deterministic=True, enable_checkpointing=False, logger=False)
        trainer.fit(model)

        result_val = get_dx_predictions(trainer, model, split='val', threshold=0.5)
        threshold = find_optimal_cutoff(result_val['true'], result_val['score'])
        #display_scores(result_val)
        result_test = get_dx_predictions(trainer, model, split='test', threshold=threshold)
        #display_scores(result_test)

        val_accuracy = metrics.balanced_accuracy_score(result_val['true'], result_val['pred'])
        test_accuracy = metrics.balanced_accuracy_score(result_test['true'], result_test['pred'])

        if val_accuracy >= best_accuracy:
            result_test.to_csv(os.path.join('results', model_name+'.csv'), index=False)

        trial.set_user_attr("val_accuracy", val_accuracy)
        trial.set_user_attr("test_accuracy", test_accuracy)
        print('TRIAL FINISHED', trial.number)
        return val_accuracy

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=seed))
    study.optimize(objective, n_trials=5)
    #joblib.dump(study, os.path.join('optuna_studies', model_name+'.pkl'))

  0%|          | 0/1 [00:00<?, ?it/s]

[I 2023-07-25 16:03:56,570] A new study created in memory with name: no-name-1baac5d3-15ed-44bf-abf9-7527e14c4c89


====EFFICIENTNETB3====
Loaded pretrained weights for efficientnet-b3


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name       | Type         | Params
--------------------------------------------
0 | loss       | BCELoss      | 0     
1 | base_model | EfficientNet | 10.7 M
2 | sigm       | Sigmoid      | 0     
--------------------------------------------
10.7 M    Trainable params
0         Non-trainable params
10.7 M    Total params
42.791    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/caduser/anaconda3/envs/overlap/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/caduser/anaconda3/envs/overlap/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

In [ ]:
study.best_trial

In [2]:
%%time
model_names = ['efficientnetb3', 'efficientnetb4.1', 'efficientnetb4.2', 'efficientnetb4.3', 'efficientnetb4.4', 'efficientnetb4.5', 
               'efficientnetb5.1', 'efficientnetb5.2', 'efficientnetb5.3', 'efficientnetb5.4', 'efficientnetb6.1', 'efficientnetb6.1', 
               'efficientnetb6.2', 'efficientnetb6.3', 'efficientnetb7.1', 'efficientnetb7.2', 'efficientnetb7.3', 'resnest101', 'se_resnext101', ]

model_names = ['efficientnetb3']

for model_name in tqdm(model_names):
    
    print(f"===={model_name}====")
    
    params = {
            'learning_rate': 2e-5, 
            'batch_size': 16
    }

    model = get_model(model_name, params=params)
    trainer = pl.Trainer(max_epochs=15, devices=1, accelerator="gpu", deterministic=True, enable_checkpointing=False, logger=False)
    trainer.fit(model)

    result_val = get_dx_predictions(trainer, model, split='val', threshold=0.5)
    threshold = find_optimal_cutoff(result_val['true'], result_val['score'])
    display_scores(result_val)
    result_test = get_dx_predictions(trainer, model, split='test', threshold=threshold)
    display_scores(result_test)
    result_test.to_csv(os.path.join('results', model_name+'.csv'), index=False)

  0%|          | 0/1 [00:00<?, ?it/s]

====efficientnetb3====
Loaded pretrained weights for efficientnet-b3


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name       | Type         | Params
--------------------------------------------
0 | loss       | BCELoss      | 0     
1 | base_model | EfficientNet | 10.7 M
2 | sigm       | Sigmoid      | 0     
--------------------------------------------
10.7 M    Trainable params
0         Non-trainable params
10.7 M    Total params
42.791    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Predicting: 168it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


=====
AUC: 0.8315278186635267
Balanced Acc: 0.7695613776861937
Sensitivity: 0.7441860465116279
Specificity: 0.7949367088607595
=====



Predicting: 168it [00:00, ?it/s]

=====
AUC: 0.8066000000000001
Balanced Acc: 0.74
Sensitivity: 0.75
Specificity: 0.73
=====

CPU times: user 4min 58s, sys: 51.5 s, total: 5min 49s
Wall time: 6min 14s
